In [1]:
import re
import pandas as pd
import unicodedata
from pathlib import Path

# Variantes de "de"
PREPOSITIONS = [
    r"\bde la\b",
    r"\bde\b",
    r"\bdu\b",
    r"\bdes\b",
    r"\bd’\b",      # apostrophe typographique
    r"\bd'\b",      # apostrophe simple
    r"\bd’un\b",
    r"\bd'une\b",
    r"\bde l’\b",
    r"\bde l'\b",
]

# Pattern pour "N de N" (avec éventuellement un champ après un "|", qu'on ignore ici)
pattern = re.compile(
    rf"(.+?)\s+(?:{'|'.join(PREPOSITIONS)})\s+(.+?)(?:\s*\|\s*(.*))?$",
    flags=re.IGNORECASE
)


def nettoyer_texte(s: str) -> str:
    """Nettoyage de base : trim, minuscules, accents, ponctuation, espaces multiples."""
    if s is None:
        return None
    s = s.strip().lower()

    # Normaliser les accents
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(c for c in s if not unicodedata.combining(c))

    # Retirer ponctuation
    s = re.sub(r"[^\w\s]", " ", s)
    # Retirer chiffres (si non pertinents)
    s = re.sub(r"\d+", "", s)
    # Espaces multiples → un seul
    s = re.sub(r"\s+", " ", s).strip()
    return s


def extraire_paires(ligne: str, relation_label: str):
    """
    Extrait (mot1, mot2) à partir d'une ligne de type `N de N ...`
    et assigne la relation passée en argument (un seul type de relation par fichier).
    """
    m = pattern.match(ligne)
    if not m:
        return None
    mot1_raw, mot2_raw, _ = m.groups()

    mot1 = nettoyer_texte(mot1_raw)
    mot2 = nettoyer_texte(mot2_raw)

    if not mot1 or not mot2:
        return None

    return mot1, mot2, relation_label


def traiter_fichier(input_path: str, relation_label: str):
    """
    Lit un fichier texte, extrait toutes les paires (mot1, mot2, relation_label)
    et renvoie une liste de dict pour ce fichier.
    """
    resultats = []
    with open(input_path, encoding="utf-8") as f:
        for lineno, ligne in enumerate(f, start=1):
            ligne = ligne.strip()
            if not ligne:
                continue
            paire = extraire_paires(ligne, relation_label)
            if paire:
                mot1, mot2, relation = paire
                resultats.append({"mot1": mot1, "mot2": mot2, "relation": relation})
            else:
                # Tu peux décommenter si tu veux voir les lignes non reconnues
                # print(f"Ligne {lineno} non extraite dans {input_path}: {ligne}")
                pass
    return resultats


def traiter_dossier(dossier_txt: str,
                    output_csv_global: str,
                    creer_csv_par_fichier: bool = False):
    """
    Parcourt tous les *.txt du dossier, utilise le nom de fichier comme label de relation
    (ex. r_loc_in.txt -> relation = 'r_loc_in'), agrège tout dans un CSV global.
    Optionnel : un CSV par fichier.
    """
    toutes_paires = []

    for chemin in Path(dossier_txt).glob("*.txt"):
        relation_label = chemin.stem  # ex. 'r_loc_in', 'r_depict', etc.

        paires = traiter_fichier(str(chemin), relation_label)
        print(f"{chemin.name}: {len(paires)} paires extraites.")

        if creer_csv_par_fichier:
            df_f = pd.DataFrame(paires)
            df_f = df_f.drop_duplicates(subset=["mot1", "mot2", "relation"]).reset_index(drop=True)
            df_f.to_csv(f"{chemin.stem}.csv", index=False, encoding="utf-8")

        toutes_paires.extend(paires)

    # CSV global
    df_global = pd.DataFrame(toutes_paires)
    df_global = df_global.drop_duplicates(subset=["mot1", "mot2", "relation"]).reset_index(drop=True)
    df_global.to_csv(output_csv_global, index=False, encoding="utf-8")
    print(f"Total: {len(df_global)} paires uniques écrites dans {output_csv_global}")


if __name__ == "__main__":
    # À adapter : chemin du dossier où se trouvent tes r_depict.txt, r_has_causitif.txt, etc.
    DOSSIER_RELATIONS = "/content/sample_data/"  # <-- change ici
    SORTIE_GLOBALE = "base_relations.csv"

    traiter_dossier(
        dossier_txt=DOSSIER_RELATIONS,
        output_csv_global=SORTIE_GLOBALE,
        creer_csv_par_fichier=True  # mets False si tu ne veux QUE le CSV global
    )


r_depict.txt: 1045 paires extraites.
r_quantificateur.txt: 880 paires extraites.
r_processusinstr-1.txt: 678 paires extraites.
r_topic.txt: 996 paires extraites.
r_lieu_origine.txt: 438 paires extraites.
r_processusagent.txt: 870 paires extraites.
r_holo.txt: 868 paires extraites.
r_object_matière.txt: 1283 paires extraites.
r_has_causitif.txt: 961 paires extraites.
r_processuspatient.txt: 1207 paires extraites.
r_own-1.txt: 1074 paires extraites.
r_social_tie.txt: 803 paires extraites.
r_has_property.txt: 1021 paires extraites.
r_product_of.txt: 1188 paires extraites.
Total: 9885 paires uniques écrites dans base_relations.csv


In [3]:
#!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 66.7 MB/s eta 0:00:00


In [4]:
# =========================
# 0. IMPORTS ET CONFIG
# =========================

import os
import json
import pickle
import numpy as np
import pandas as pd

from gensim.models import Word2Vec
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from numpy.linalg import norm
from collections import defaultdict


In [5]:
# =========================
# 1. CHARGER LA BASE GLOBALE
# =========================

CSV_PATH = "base_relations.csv"  # adapter si besoin

df = pd.read_csv(CSV_PATH)
print("Aperçu des données :")
print(df.head())

relations_uniques = sorted(df["relation"].unique())
print("\nRelations uniques :", relations_uniques)
print("Nombre total de paires :", len(df))



Aperçu des données :
       mot1      mot2  relation
0     photo   famille  r_depict
1  peinture   paysage  r_depict
2   gravure  monument  r_depict
3   fresque  bataille  r_depict
4  mosaique      dieu  r_depict

Relations uniques : ['r_depict', 'r_has_causitif', 'r_has_property', 'r_holo', 'r_lieu_origine', 'r_object_matière', 'r_own-1', 'r_processusagent', 'r_processusinstr-1', 'r_processuspatient', 'r_product_of', 'r_quantificateur', 'r_social_tie', 'r_topic']
Nombre total de paires : 9885


In [6]:
# =========================
# 2. FONCTIONS UTILITAIRES
# =========================

def get_vec(word, model, dim):
    """
    Retourne le vecteur Word2Vec du mot s'il existe, sinon un vecteur nul.
    """
    if word in model.wv:
        return model.wv[word]
    else:
        return np.zeros(dim, dtype=np.float32)


def combine(v1, v2):
    """
    Combine deux vecteurs v1, v2 en un vecteur de paire :
    [v1, v2, |v1 - v2|, v1 * v2]
    """
    v1 = np.asarray(v1, dtype=np.float32)
    v2 = np.asarray(v2, dtype=np.float32)
    diff = np.abs(v1 - v2)
    prod = v1 * v2
    return np.concatenate([v1, v2, diff, prod], axis=-1)


def encode_pair(mot1, mot2, w2v_model, emb_dim):
    """
    Encode un couple (mot1, mot2) en vecteur combiné avec un modèle Word2Vec donné.
    """
    v1 = get_vec(mot1, w2v_model, emb_dim)
    v2 = get_vec(mot2, w2v_model, emb_dim)
    return combine(v1, v2)


In [7]:
# =========================
# 3. ENTRAÎNER LES MODÈLES PAR RELATION
# =========================

emb_dim = 100     # dimension des embeddings Word2Vec
neg_ratio = 1.0   # négatifs ≈ positifs (1.0 = autant de négatifs que de positifs)

models_R = {}     # relation -> {"w2v": w2v_R, "tree": tree_R, "emb_dim": emb_dim}

for R in relations_uniques:
    print(f"\n=== Relation {R} ===")

    # 3.1 POSITIFS : toutes les paires avec relation R
    df_pos = df[df["relation"] == R].reset_index(drop=True)
    n_pos = len(df_pos)
    print(f"Nb positifs (R) :", n_pos)

    if n_pos == 0:
        print("Aucun positif pour cette relation, on saute.")
        continue

    # 3.2 NÉGATIFS : paires avec relation != R (échantillon)
    df_neg_all = df[df["relation"] != R]
    n_neg_cible = int(neg_ratio * n_pos)
    n_neg_cible = min(n_neg_cible, len(df_neg_all))

    df_neg = df_neg_all.sample(n=n_neg_cible, random_state=42).reset_index(drop=True)
    print(f"Nb négatifs (pas R) :", len(df_neg))

    # 3.3 Corpus pour Word2Vec_R : uniquement les positifs (mot1, mot2) de R
    sentences_R = df_pos[["mot1", "mot2"]].values.tolist()

    w2v_R = Word2Vec(
        sentences=sentences_R,
        vector_size=emb_dim,
        window=5,
        min_count=1,
        workers=4,
        sg=1
    )

    # 3.4 Construire les vecteurs combinés POSITIFS
    X_pos = []
    for _, row in df_pos.iterrows():
        v_pair = encode_pair(row["mot1"], row["mot2"], w2v_R, emb_dim)
        X_pos.append(v_pair)
    X_pos = np.stack(X_pos)
    y_pos = np.ones(len(X_pos), dtype=np.int64)   # label 1 = "est R"

    # 3.5 Construire les vecteurs combinés NÉGATIFS
    X_neg = []
    for _, row in df_neg.iterrows():
        v_pair = encode_pair(row["mot1"], row["mot2"], w2v_R, emb_dim)
        X_neg.append(v_pair)
    X_neg = np.stack(X_neg)
    y_neg = np.zeros(len(X_neg), dtype=np.int64)  # label 0 = "pas R"

    # 3.6 Dataset final pour la relation R
    X_R = np.vstack([X_pos, X_neg])
    y_R = np.concatenate([y_pos, y_neg])

    # 3.7 Split train / val
    X_train, X_val, y_train, y_val = train_test_split(
        X_R, y_R, test_size=0.2, stratify=y_R, random_state=42
    )

    # 3.8 Arbre de décision binaire pour R
    tree_R = DecisionTreeClassifier(
        max_depth=15,
        min_samples_leaf=5,
        class_weight="balanced",
        random_state=42
    )
    tree_R.fit(X_train, y_train)

    val_acc = tree_R.score(X_val, y_val)
    print(f"Accuracy val pour {R} : {val_acc:.3f}")

    # 3.9 Stocker le modèle
    models_R[R] = {
        "w2v": w2v_R,
        "tree": tree_R,
        "emb_dim": emb_dim,
        "n_pos": int(n_pos),
        "n_neg": int(len(df_neg))
    }

print("\nNombre de relations effectivement entraînées :", len(models_R))



=== Relation r_depict ===
Nb positifs (R) : 977
Nb négatifs (pas R) : 977
Accuracy val pour r_depict : 0.967

=== Relation r_has_causitif ===
Nb positifs (R) : 739
Nb négatifs (pas R) : 739
Accuracy val pour r_has_causitif : 1.000

=== Relation r_has_property ===
Nb positifs (R) : 821
Nb négatifs (pas R) : 821
Accuracy val pour r_has_property : 0.985

=== Relation r_holo ===
Nb positifs (R) : 690
Nb négatifs (pas R) : 690
Accuracy val pour r_holo : 0.996

=== Relation r_lieu_origine ===
Nb positifs (R) : 365
Nb négatifs (pas R) : 365
Accuracy val pour r_lieu_origine : 0.959

=== Relation r_object_matière ===
Nb positifs (R) : 600
Nb négatifs (pas R) : 600
Accuracy val pour r_object_matière : 0.971

=== Relation r_own-1 ===
Nb positifs (R) : 388
Nb négatifs (pas R) : 388
Accuracy val pour r_own-1 : 0.987

=== Relation r_processusagent ===
Nb positifs (R) : 513
Nb négatifs (pas R) : 513
Accuracy val pour r_processusagent : 0.995

=== Relation r_processusinstr-1 ===
Nb positifs (R) : 592

In [8]:
# =========================
# 4. SAUVEGARDE DES MODÈLES
# =========================

MODEL_DIR = "models_relations"
os.makedirs(MODEL_DIR, exist_ok=True)

# 4.1 Sauvegarde de chaque modèle par relation
for R, obj in models_R.items():
    w2v_R = obj["w2v"]
    tree_R = obj["tree"]
    emb_dim_R = obj["emb_dim"]

    w2v_path = os.path.join(MODEL_DIR, f"w2v_{R}.model")
    tree_path = os.path.join(MODEL_DIR, f"tree_{R}.pkl")

    # Word2Vec
    w2v_R.save(w2v_path)

    # arbre de décision
    with open(tree_path, "wb") as f:
        pickle.dump(tree_R, f)

    print(f"Sauvé : {w2v_path} et {tree_path}")

# 4.2 Sauvegarde des métadonnées globales
metadata = {
    "relations": list(models_R.keys()),
    "emb_dim": emb_dim
}

meta_path = os.path.join(MODEL_DIR, "metadata.json")
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f"\nMétadonnées sauvegardées dans {meta_path}")


Sauvé : models_relations/w2v_r_depict.model et models_relations/tree_r_depict.pkl
Sauvé : models_relations/w2v_r_has_causitif.model et models_relations/tree_r_has_causitif.pkl
Sauvé : models_relations/w2v_r_has_property.model et models_relations/tree_r_has_property.pkl
Sauvé : models_relations/w2v_r_holo.model et models_relations/tree_r_holo.pkl
Sauvé : models_relations/w2v_r_lieu_origine.model et models_relations/tree_r_lieu_origine.pkl
Sauvé : models_relations/w2v_r_object_matière.model et models_relations/tree_r_object_matière.pkl
Sauvé : models_relations/w2v_r_own-1.model et models_relations/tree_r_own-1.pkl
Sauvé : models_relations/w2v_r_processusagent.model et models_relations/tree_r_processusagent.pkl
Sauvé : models_relations/w2v_r_processusinstr-1.model et models_relations/tree_r_processusinstr-1.pkl
Sauvé : models_relations/w2v_r_processuspatient.model et models_relations/tree_r_processuspatient.pkl
Sauvé : models_relations/w2v_r_product_of.model et models_relations/tree_r_pro

In [9]:
# =========================
# 5. RECHARGEMENT DES MODÈLES
# =========================

MODEL_DIR = "models_relations"

# 5.1 Charger les métadonnées
meta_path = os.path.join(MODEL_DIR, "metadata.json")
with open(meta_path, "r", encoding="utf-8") as f:
    metadata = json.load(f)

relations_loaded = metadata["relations"]
emb_dim_loaded = metadata["emb_dim"]

print("Relations chargées :", relations_loaded)
print("Dimension d'embedding :", emb_dim_loaded)

# 5.2 Reconstruire models_R_loaded
models_R_loaded = {}

for R in relations_loaded:
    w2v_path = os.path.join(MODEL_DIR, f"w2v_{R}.model")
    tree_path = os.path.join(MODEL_DIR, f"tree_{R}.pkl")

    # Word2Vec_R
    w2v_R = Word2Vec.load(w2v_path)

    # arbre R
    with open(tree_path, "rb") as f:
        tree_R = pickle.load(f)

    models_R_loaded[R] = {
        "w2v": w2v_R,
        "tree": tree_R,
        "emb_dim": emb_dim_loaded
    }

print("Nombre de modèles rechargés :", len(models_R_loaded))


Relations chargées : ['r_depict', 'r_has_causitif', 'r_has_property', 'r_holo', 'r_lieu_origine', 'r_object_matière', 'r_own-1', 'r_processusagent', 'r_processusinstr-1', 'r_processuspatient', 'r_product_of', 'r_quantificateur', 'r_social_tie', 'r_topic']
Dimension d'embedding : 100
Nombre de modèles rechargés : 14


In [10]:
# =========================
# 6. PRÉDICTION POUR UNE NOUVELLE PAIRE
# =========================

def predict_relation_multi_R(mot1, mot2, models_R, seuil=0.0):
    """
    Prédit la relation pour (mot1, mot2) en testant tous les modèles par relation.

    mot1, mot2 : strings
    models_R : dict[relation] -> {"w2v": w2v_R, "tree": tree_R, "emb_dim": emb_dim}
    seuil : seuil minimal de probabilité pour accepter une relation.

    Retourne :
      - relation_finale (str ou None si rejet)
      - score_max (float)
      - scores_par_relation (dict[relation] -> score)
    """
    scores = {}

    for R, obj in models_R.items():
        w2v_R = obj["w2v"]
        tree_R = obj["tree"]
        emb_dim = obj["emb_dim"]

        v_pair = encode_pair(mot1, mot2, w2v_R, emb_dim).reshape(1, -1)

        if hasattr(tree_R, "predict_proba"):
            proba = tree_R.predict_proba(v_pair)[0, 1]  # proba classe 1 = "est R"
        else:
            pred = tree_R.predict(v_pair)[0]
            proba = float(pred)

        scores[R] = float(proba)

    # relation avec probabilité max
    relation_max = max(scores, key=scores.get)
    score_max = scores[relation_max]

    if score_max < seuil:
        # si aucune relation n'est jugée suffisamment probable
        return None, score_max, scores

    return relation_max, score_max, scores


In [14]:
# Exemple de de
mot1_test = "desert"
mot2_test = "frankfurt"

relation_predite, score_max, scores_detail = predict_relation_multi_R(
    mot1_test,
    mot2_test,
    models_R_loaded,   # ou models_R si tu es encore dans le même notebook
    seuil=0.0          # tu peux mettre 0.4 ou 0.5 pour être plus strict
)

print("Mot1 :", mot1_test)
print("Mot2 :", mot2_test)
print("Relation prédite :", relation_predite)
print("Score max :", score_max)
print("Scores par relation :")
for r, s in scores_detail.items():
    print(f"  {r}: {s:.3f}")


Mot1 : desert
Mot2 : frankfurt
Relation prédite : r_social_tie
Score max : 0.0021008403361344537
Scores par relation :
  r_depict: 0.000
  r_has_causitif: 0.000
  r_has_property: 0.000
  r_holo: 0.000
  r_lieu_origine: 0.000
  r_object_matière: 0.000
  r_own-1: 0.000
  r_processusagent: 0.000
  r_processusinstr-1: 0.000
  r_processuspatient: 0.000
  r_product_of: 0.000
  r_quantificateur: 0.000
  r_social_tie: 0.002
  r_topic: 0.000
